In [1]:
import pickle
import io

In [2]:
all_data = pickle.load(open("all_14sub/all_14sub.p", "rb" ),encoding='iso-8859-1')
print(type(all_data), all_data.shape, all_data[:, -1])

<class 'numpy.ndarray'> (1750000, 23) [ 2.  2.  2. ... 17. 17. 17.]


In [3]:
import tensorflow as tf
import numpy as np
import time
import pickle
from sklearn import preprocessing

In [4]:
def one_hot(y_):
    # Function to encode output labels from number indexes
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    y_ = y_.reshape(len(y_))
    y_ = [int(x) for x in y_]
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]


In [5]:
def extract(input, n_fea, time_window, moving):
    global n_classes
    xx = input[:, :n_fea]
    yy = input[:, n_fea:n_fea + 1]
    new_x = []
    new_y = []
    number = int((xx.shape[0] / moving) - 1)
    for i in range(number):
        ave_y = np.average(yy[int(i * moving):int(i * moving + time_window)])
        if ave_y in range(n_classes + 1):
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(ave_y)
        else:
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(0)

    new_x = np.array(new_x)
    new_x = new_x.reshape([-1, n_fea * time_window])
    new_y = np.array(new_y)
    new_y.shape = [new_y.shape[0], 1]
    data = np.hstack((new_x, new_y))
    data = np.vstack((data, data[-1]))  # add the last sample again, to make the sample number round
    return data

In [6]:
def compute_accuracy_t(v_xs, v_ys):  # this function only calculate the acc of CNN_task
    global prediction_t
    y_pre = sess.run(prediction_t, feed_dict={xs: v_xs, keep_prob: keep})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys_t: v_ys, keep_prob: keep})
    return result

In [7]:
def compute_accuracy_p(v_xs, v_ys):  # this function only calculate the acc of CNN_task
    global prediction_p
    y_pre = sess.run(prediction_p, feed_dict={xs: v_xs, keep_prob: keep})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys_p: v_ys, keep_prob: keep})
    return result

In [8]:
n_classes = 2
n_person_ = 13  # the number of training subjects
sample_persub = 250*500  # we have overlapping now
print(type(all_data), all_data.shape, all_data[:, -1])

<class 'numpy.ndarray'> (1750000, 23) [ 2.  2.  2. ... 17. 17. 17.]


In [9]:
no_fea = 21  # data.shape[-1] - 1
seg_length = 250  # # 255 for raw data, 96 for layer 23, 64 for layer 2, 32 for layer 2

scaler = preprocessing.MinMaxScaler()  # normalization
F = scaler.fit_transform(all_data[:, :no_fea])  # scale to [0, 1]

all_data = np.hstack((F, all_data[:, no_fea:no_fea+1]))  # only use the task ID


"""Make person label"""
n_sample_ = int(2*sample_persub/seg_length )  # the number of sampls of each subject after reshape
ll = np.ones([n_sample_, 1])*0
for hh in range(1, n_person_):
    ll_new = np.ones([n_sample_, 1])*hh
    ll = np.vstack((ll, ll_new))
print('the shape of maked person label', ll.shape)


the shape of maked person label (13000, 1)


In [10]:
import time

In [11]:
n_classes = 2
n_person_ = 13  # the number of training subjects
sample_persub = 250*500  # we have overlapping now
print(type(all_data), all_data.shape, all_data[:, -1])

no_fea = 21  # data.shape[-1] - 1
seg_length = 250  # # 255 for raw data, 96 for layer 23, 64 for layer 2, 32 for layer 2

scaler = preprocessing.MinMaxScaler()  # normalization
F = scaler.fit_transform(all_data[:, :no_fea])  # scale to [0, 1]

all_data = np.hstack((F, all_data[:, no_fea:no_fea+1]))  # only use the task ID


"""Make person label"""
n_sample_ = int(2*sample_persub/seg_length )  # the number of sampls of each subject after reshape
ll = np.ones([n_sample_, 1])*0
for hh in range(1, n_person_):
    ll_new = np.ones([n_sample_, 1])*hh
    ll = np.vstack((ll, ll_new))
print('the shape of maked person label', ll.shape)

ll_test = np.ones([n_sample_, 1])*n_person_


<class 'numpy.ndarray'> (1750000, 22) [0. 0. 0. ... 1. 1. 1.]
the shape of maked person label (13000, 1)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [41]:
rf_auc_list = []
svm_auc_list = []
knn_auc_list = []
for P_ID in range(14):  # n_person_++1
    if P_ID==0:
        reuse=False
    else:
        reuse=True
    """Select train and test subject"""
    data_ = all_data[sample_persub*P_ID:sample_persub*(P_ID+1)]

    list = range(sample_persub*P_ID, sample_persub*(P_ID+1))
    data = np.delete(all_data, list, axis=0)
    # overlap
    train_data = extract(data, n_fea=no_fea, time_window=seg_length, moving=(seg_length/2))
    test_data = extract(data_, n_fea=no_fea, time_window=seg_length, moving=(seg_length/2))  # 50% overlapping
    # continue
    """Replace the original person data by the maked data"""
    no_fea_long = train_data.shape[-1] - 1  # here is - 2, because has two IDs

    train_data = np.hstack((train_data[:, :no_fea_long+1], ll))
    test_data = np.hstack((test_data[:, :no_fea_long + 1], ll_test))

    np.random.shuffle(train_data)
    np.random.shuffle(test_data)


    feature_train = train_data[:, :no_fea_long]
    feature_test = test_data[:, :no_fea_long]
    label_train_t = train_data[:, no_fea_long:no_fea_long + 1]
    label_test_t = test_data[:, no_fea_long:no_fea_long + 1]
    label_train_p = train_data[:, no_fea_long + 1:no_fea_long + 2]

    label_train_t = one_hot(label_train_t)
    label_test_t = one_hot(label_test_t)
    label_train_p = one_hot(label_train_p)
    

    rf = RandomForestClassifier(n_estimators=50)
    labels = label_train_t[:,0]
    rf.fit(feature_train,labels)

    preds = rf.predict_proba(feature_test)
    preds = preds[:,1]
    preds = [*preds]
    
    print(metrics.roc_auc_score(label_test_t[:,0],preds))
    rf_auc_list.append((metrics.roc_auc_score(label_test_t[:,0],preds)))
    
    svm_model = svm.SVC(C = 1.0, kernel = 'linear', probability = True)
    svm_model.fit(feature_train, labels)
    
    preds = svm_model.predict_proba(feature_test)
    preds = preds[:,1]
    preds = [*preds]
    
    print(metrics.roc_auc_score(label_test_t[:,0],preds))
    svm_auc_list.append((metrics.roc_auc_score(label_test_t[:,0],preds)))
    
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(feature_train, labels)
    
    preds = knn.predict_proba(feature_test)
    
    preds = preds[:,1]
    preds = [*preds]
    
    print(metrics.roc_auc_score(label_test_t[:,0],preds))
    knn_auc_list.append((metrics.roc_auc_score(label_test_t[:,0],preds)))

0.637354921797418
0.4338924404170804
0.6519403239821252
0.9156229166666667
0.5600666666666667
0.6715666666666666
0.9871580242673993
0.44442536630036633
0.7479073660714286
0.8733030174471019
0.47268476781389757
0.5874144879885453
0.5567462422634837
0.5958974358974358
0.48318302387267903
0.9021625000000001
0.45823749999999996
0.5449208333333333
0.7784925312980175
0.4696151036550039
0.6544434368623895
0.7335393101522134
0.5552646278452731
0.6253534771679933
0.5934665723778736
0.5189194953304598
0.4804743624281609
0.8543
0.4917416666666667
0.5912125
0.9160314479304759
0.5612019443216968
0.7390862056267491
0.6630083333333334
0.5148333333333334
0.6041583333333334
0.7438881802721088
0.43773030045351474
0.5652695105820105
0.7041029299093815
0.5730429843333069
0.5151085836569708


In [43]:
print(np.mean(rf_auc_list))
print(np.mean(svm_auc_list))
print(np.mean(knn_auc_list))

0.7756554948368196
0.5062538309310503
0.6044313651123133
